<a href="https://colab.research.google.com/github/theclosedbook/TensorFlow/blob/main/05_Transfer_Learning_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with TensorFlow Part 2: Fine-Tuning
In Previous notebook, we covered transfer learning feature extraction, now it's time to learn about a new kind of transfer learning: Fine-Tuning

In [2]:
!nvidia-smi

Mon Feb 19 19:32:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Creating Helper functions

In previous notebooks, we've created a bunch of helper functions.

So it's a good idea to put functions you'll want to use again in a script you can download and import into your notebooks(or elsewhere).



In [4]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-02-19 19:34:47--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-02-19 19:34:47 (118 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [5]:
# Import helper functions we're going to use in this notebook
from helper_functions import create_tensorboard_callback,plot_loss_curves,unzip_data,walk_through_dir

### Let's Get some data

This time we're going to see how we can use the pretrained models within `tf.keras.application` and apply
them to our own problems (recognizing images of food)

In [8]:
# Get 10% of the training data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
unzip_data("10_food_classes_10_percent.zip")

--2024-02-19 19:43:10--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.207, 74.125.68.207, 64.233.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.2’

10_food_classes_10_ 100%[===================>] 160.74M  22.4MB/s    in 8.4s    

2024-02-19 19:43:18 (19.2 MB/s) - ‘10_food_classes_10_percent.zip.2’ saved [168546183/168546183]



In [17]:
# Check out how many images and subdirecteries are in our dataset
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/tes

In [21]:
# Create training and testing directory paths
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

In [22]:
import tensorflow as tf
IMG_SIZE=(224,224)
BATCH_SIZE=32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size = IMG_SIZE,
                                                                            label_mode="categorical",
                                                                            batch_size=BATCH_SIZE)
test_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                           image_size=IMG_SIZE,
                                                                           label_mode="categorical",
                                                                           batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [23]:
train_data_10_percent

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [24]:
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [25]:
for images,labels in train_data_10_percent.take(1):
  print(images,labels)

tf.Tensor(
[[[[4.3000000e+01 3.1000000e+01 1.8285715e+01]
   [4.6714287e+01 3.4714287e+01 2.2714285e+01]
   [4.2704079e+01 3.1132652e+01 1.9132652e+01]
   ...
   [2.5414288e+02 2.5314288e+02 2.2471436e+02]
   [2.5492856e+02 2.5292856e+02 2.2792856e+02]
   [2.5264282e+02 2.5064282e+02 2.2564282e+02]]

  [[3.7188774e+01 2.4188774e+01 1.5188775e+01]
   [3.6423470e+01 2.3423468e+01 1.4423468e+01]
   [4.1428574e+01 2.9071428e+01 1.9857141e+01]
   ...
   [2.5285716e+02 2.5185716e+02 2.2342863e+02]
   [2.5299998e+02 2.5099998e+02 2.2599998e+02]
   [2.5164282e+02 2.4964282e+02 2.2464282e+02]]

  [[3.7780613e+01 2.4704081e+01 1.7923470e+01]
   [3.4270405e+01 2.1270409e+01 1.3698979e+01]
   [3.6382652e+01 2.3290817e+01 1.5765306e+01]
   ...
   [2.5238263e+02 2.5138263e+02 2.2116835e+02]
   [2.5200000e+02 2.5100000e+02 2.2257143e+02]
   [2.5200000e+02 2.5100000e+02 2.2257143e+02]]

  ...

  [[2.4049495e+02 2.0849495e+02 1.7028069e+02]
   [2.3434184e+02 2.0234184e+02 1.6412758e+02]
   [2.2157138e+